# Verified Integer Mathematics in Transformers - Describe Model Algorithm

This Colab describes the algorithm of Transformer models in terms of model behaviours and algorithmic sub-tasks (analysed in other Colabs and stored in JSON files).

The models that performs integer addition, subtraction and/or multiplication e.g. 133357+182243=+0315600, 123450-345670=-0123230 and 000345*000823=+283935. Each digit is a separate token. For 6 digit questions, the model is given 14 "question" (input) tokens, and must then predict the corresponding 8 "answer" (output) tokens.

This Colab follows on from:
- https://github.com/PhilipQuirke/verified_transformers/blob/main/notebooks/VerifiedArithmeticTrain.ipynb trained the models. It outputs model_name.pth and model_name_train.json
- https://github.com/PhilipQuirke/verified_transformers/blob/main/notebooks/VerifiedArithmeticAnalyse.ipynb analyzes the models. It documents their sub-tasks in model_name_behavior.json model_name_algorithm.json

This Colab loads the above json files from HuggingFace repository https://huggingface.co/PhilipQuirke/VerifiedArithmetic/tree/main


## Tips for using the Colab
 * You can run and alter the code in this CoLab notebook yourself in Google CoLab ( https://colab.research.google.com/ ).
 * To run the notebook, in Google CoLab, **you will need to** go to Runtime > Change Runtime Type and select GPU as the hardware accelerator.
 * Some graphs are interactive!
 * Use the table of contents pane in the sidebar to navigate.
 * Collapse irrelevant sections with the dropdown arrows.
 * Search the page using the search in the sidebar, not CTRL+F.

# Part 0: Import libraries
Imports standard libraries.

Imports "verified_transformer" public library as "qt". This library is specific to this CoLab's "QuantaTool" approach to transformer analysis. Refer to [README.md](https://github.com/PhilipQuirke/verified_transformers/blob/main/README.md) for more detail.

In [ ]:
DEVELOPMENT_MODE = True
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    !pip install matplotlib

    !pip install kaleido
    !pip install transformer_lens
    !pip install torchtyping

    !pip install numpy
    #!pip install scikit-learn
    !pip install huggingface_hub

except:
    IN_COLAB = False

    def setup_jupyter(install_libraries=False):
        if install_libraries:
            !pip install matplotlib==3.8.4
            !pip install kaleido==0.2.1
            !pip install transformer_lens==1.15.0
            !pip install torchtyping==0.1.4
            !pip install numpy==1.26.4
            !pip install plotly==5.20.0
            !pip install pytest==8.1.1
            #!pip install scikit-learn==1.4.1.post1

        print("Running as a Jupyter notebook - intended for development only!")
        from IPython import get_ipython

        ipython = get_ipython()
        # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
        ipython.magic("load_ext autoreload")
        ipython.magic("autoreload 2")

    # setup_jupyter(install_libraries=True)   # Uncomment if you need to install libraries in notebook.
    setup_jupyter(install_libraries=False)

In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import kaleido
import plotly.io as pio

if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

In [ ]:
pio.templates['plotly'].layout.xaxis.title.font.size = 20
pio.templates['plotly'].layout.yaxis.title.font.size = 20
pio.templates['plotly'].layout.title.font.size = 30

In [ ]:
import json
import torch
import torch.nn.functional as F
import numpy as np
import random
import itertools
import re
from enum import Enum

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import textwrap
from huggingface_hub import hf_hub_download

In [ ]:
! pip uninstall QuantaTools -y || true   # Ensure a clean install.

In [ ]:
# Refer https://github.com/PhilipQuirke/verified_transformers/blob/main/README.md
!pip install --upgrade git+https://github.com/PhilipQuirke/verified_transformers.git  # Specify @branch if testing a specific branch
import QuantaTools as qt

# Part 1A: Configuration

Which existing model do we want to analyse?

The existing model weightings created by the sister Colab [VerifiedArithmeticTrain](https://github.com/PhilipQuirke/transformer-maths/blob/main/assets/VerifiedArithmeticTrain.ipynb) are loaded from HuggingFace (in Part 5). Refer https://github.com/PhilipQuirke/verified_transformers/blob/main/README.md for more detail.

In [ ]:
# Singleton QuantaTool "main" configuration class. MathsConfig is derived from the chain AlgoConfig > UsefulConfig > ModelConfig
cfg = qt.MathsConfig()

# Singleton QuantaTool "ablation intervention" configuration class
acfg = qt.acfg

In [ ]:
# Which model do we want to analyse? Uncomment one line:

#cfg.model_name = "" # Use configuration specified in cfg defaults

#cfg.model_name = 5-digit and 6-digit digit Addition models
#cfg.model_name = "add_d5_l1_h3_t15K_s372001"  # Inaccurate as only has one layer. Can predict S0, S1 and S2 complexity questions.
#cfg.model_name = "add_d5_l2_h3_t15K_s372001"  # AvgFinalLoss=1.6e-08. Accurate on 1M Qs
#cfg.model_name = "add_d6_l2_h3_t15K_s372001"  # AvgFinalLoss=1.7e-08. Accurate on 1M Qs. MAIN FOCUS
#cfg.model_name = "add_d6_l2_h3_t20K_s173289"  # AvgFinalLoss=1.5e-08. Accurate on 1M Qs
#cfg.model_name = "add_d6_l2_h3_t20K_s572091"  # AvgFinalLoss=7e-09. Accurate on 1M Qs

# 6-digit Subtraction model
#cfg.model_name = "sub_d6_l2_h3_t30K_s372001"  # AvgFinalLoss=5.8e-06. Fails 1M Qs

# 6-digit Mixed (addition and subtraction) models
#cfg.model_name = "mix_d6_l3_h4_t40K_s372001"  # AvgFinalLoss=5e-09. Fails 1M Qs

# "ins1" 6-digit Mixed models initialised with 6-digit addition model
cfg.model_name = "ins1_mix_d6_l3_h4_t40K_s372001"  # AvgFinalLoss=8e-09. Accurate on 1M Qs for Add and Sub. MAIN FOCUS
#cfg.model_name = "ins1_mix_d6_l3_h4_t40K_s173289"  # AvgFinalLoss=1.6e-08. 936K for Add, 1M Qs for Sub
#cfg.model_name = "ins1_mix_d6_l3_h4_t50K_s572091"  # AvgFinalLoss=2.9e-08. 1M for Add. 300K for Sub. For 000041-000047=-0000006 gives +0000006. Improve training data.
#cfg.model_name = "ins1_mix_d6_l3_h3_t40K_s572091"  #  AvgFinalLoss=1.8e-08. Fails on 1M Qs. For 099111-099111=+0000000 gives -0000000. Improve training data.

# "ins2" 6-digit Mixed model initialised with 6-digit addition model. Reset useful heads every 100 epochs.
#cfg.model_name = "ins2_mix_d6_l4_h4_t40K_s372001"  # AvgFinalLoss=7e-09. Fails 1M Qs

# "ins3" 6-digit Mixed model initialised with 6-digit addition model. Reset useful heads & MLPs every 100 epochs.
#cfg.model_name = "ins3_mix_d6_l4_h3_t40K_s372001"  # AvgFinalLoss=2.6e-06. Fails 1M Qs

# Part 1B: Configuration: Input and Output file names



In [ ]:
# Needed when user changes model_name and reruns this Colab a second time
cfg.reset_useful()
cfg.reset_algo()
cfg.initialize_maths_token_positions()
acfg.reset_ablate()

if cfg.model_name != "":
  # Update cfg member data n_digits, n_layers, n_heads, n_training_steps from model_name
  cfg.parse_model_name()

  # Addition model
  cfg.perc_sub = 0
  if cfg.model_name.startswith("sub_") :
    # Subtraction model
    cfg.perc_sub = 100
  elif cfg.model_name.startswith("mix") :
    # Mixed (addition and subtraction) model
    cfg.perc_sub = 66 # Train on 66% subtraction and 33% addition question batches
  elif cfg.model_name.startswith("ins") :
    # Mixed model initialised with an addition model (using insert mode 1, 2 or 3)
    cfg.perc_sub = 80 # Train on 80% subtraction and 20% addition question batches

  # We train multiple versions of some models, inserting different addition models.
  cfg.insert_training_seed = cfg.training_seed

  if cfg.model_name.startswith("ins1_mix_d6_l3") :
    if cfg.training_seed == 372001:
      # Mixed model initialised with add_d6_l2_h3_t15K.pth.
      cfg.insert_n_training_steps = 15000
    else:
      # Mixed model initialised with add_d6_l2_h3_t20K.pth.
      cfg.insert_n_training_steps = 20000

  cfg.batch_size = 512 # Default analysis batch size
  if cfg.n_layers >= 3 and cfg.n_heads >= 4:
    cfg.batch_size = 256 # Reduce batch size to avoid memory constraint issues.

In [ ]:
main_fname = cfg.file_config_prefix()
main_fname_pth = main_fname + '.pth'
main_fname_behavior_json = main_fname + '_behavior.json'
main_fname_algorithm_json = main_fname + '_algorithm.json'

def print_config():
  print("%Add=", cfg.perc_add(), "%Sub=", cfg.perc_sub, "%Mult=", cfg.perc_mult, "InsertMode=", cfg.insert_mode, "File=", main_fname)

print_config()
print("weight_decay=", cfg.weight_decay, "lr=", cfg.lr, "batch_size=", cfg.batch_size)
print('Main model will be read from HuggingLab file', main_fname_pth)
print('Main model behavior analysis tags will be read from HuggingLab file', main_fname_behavior_json)
print('Main model sub-task analysis tags will be read from HuggingLab file', main_fname_algorithm_json)

# Part 3A: Set Up: Vocabulary / Embedding / Unembedding

  

In [ ]:
main_fname_pth

In [ ]:
qt.set_maths_vocabulary(cfg)
qt.set_maths_question_meanings(cfg)
print(cfg.token_position_meanings)

# Part 6A: Set Up: Load nodes and behaviour tags
Load the useful nodes and associated behaviour tags from a JSON file stored on HuggingFace

In [ ]:
main_repo_name="PhilipQuirke/VerifiedArithmetic"
print("Loading useful node list with behavior tags from HuggingFace", main_repo_name, main_fname_behavior_json)

file_path = hf_hub_download(repo_id=main_repo_name, filename=main_fname_behavior_json, revision="main")

cfg.useful_nodes.load_nodes(file_path)

# Part 6B: Results: Show nodes and behaviour tags

In [ ]:
cfg.useful_nodes.sort_nodes()
cfg.useful_nodes.print_node_tags()

In [ ]:
def show_quanta_map( title, standard_quanta, cell_num_shades, \
        filters : qt.FilterNode, major_tag : qt.QType, minor_tag : str, get_node_details,  \
        image_width_inches : int = -1, image_height_inches : int = -1,
        combine_identical_cells : bool = True, cell_fontsize : int = 9 ): \

  test_nodes = cfg.useful_nodes
  if filters is not None:
    test_nodes = qt.filter_nodes(test_nodes, filters)

  ax1, quanta_results, num_results = qt.calc_quanta_map(
      cfg, standard_quanta, cell_num_shades,
      test_nodes, major_tag.value, minor_tag, get_node_details,
      cell_fontsize, combine_identical_cells, image_width_inches, image_height_inches )

  if num_results > 0:
    plt.show()

# Part 16A: Results: Show failure percentage map

Show the percentage failure rate (incorrect prediction) when individual Attention Heads and MLPs are ablated.

In [ ]:
show_quanta_map( "Failure Frequency Behavior Per Node", True, qt.FAIL_SHADES, None, qt.QType.FAIL, "", qt.get_quanta_fail_perc, 9, 2 * cfg.n_layers, False)

# Part 16B - Show answer impact behavior map

Show the purpose of each useful cell by impact on the answer digits A0 to Amax.


In [ ]:
show_quanta_map( "Answer Impact Behavior Per Node", True, cfg.num_answer_positions, None, qt.QType.IMPACT, "", qt.get_quanta_impact, 9, 2 * cfg.n_layers)

# Part 16C: Result: Show attention map

Show attention quanta of useful heads

In [ ]:
# Only maps attention heads, not MLP layers
show_quanta_map( "Attention Behavior Per Head", True, qt.ATTN_SHADES, None, qt.QType.ATTN, "", qt.get_quanta_attention, 9, 3 * cfg.n_layers)

# Part 16C - Show question complexity map


In [ ]:
if cfg.perc_sub > 0:
  # For each useful cell, show if addition (S), positive-answer subtraction (M) and negative-answer subtraction (N) questions relies on the node.
  show_quanta_map( "Maths Operation Coverage", False,
      4, None, qt.QType.MATH, "", qt.get_maths_operation_complexity, 9, 6,
      combine_identical_cells=False)

In [ ]:
if cfg.perc_add() > 0:
  # For each useful cell, show the minimum addition question complexity that relies on the node, as measured using quanta S0, S1, S2, ...
  show_quanta_map( "Addition Min-Complexity Behavior Per Node", False,
      qt.MATH_ADD_SHADES, None, qt.QType.MATH_ADD, qt.MathsBehavior.ADD_COMPLEXITY_PREFIX.value, qt.get_maths_min_complexity, 9, 4)

In [ ]:
if cfg.perc_sub > 0:
  # For each useful cell, show the minimum "positive-answer subtraction" question complexity that relies on the node, as measured using quanta M0, M1, M2, ...
  show_quanta_map( "Positive-answer Subtraction Min-Complexity Per Node", False,
      qt.MATH_SUB_SHADES, None, qt.QType.MATH_SUB, qt.MathsBehavior.SUB_COMPLEXITY_PREFIX.value, qt.get_maths_min_complexity, 9, 6)

In [ ]:
if cfg.perc_sub > 0:
  # For each useful cell, show the minimum "negative-answer subtraction" question complexity that relies on the node, as measured using quanta N0, N1, N2, ...
  show_quanta_map( "Negative-answer Subtraction Min-Complexity Per Node", False,
      qt.MATH_SUB_SHADES, None, qt.QType.MATH_NEG, qt.MathsBehavior.NEG_COMPLEXITY_PREFIX.value, qt.get_maths_min_complexity, 9, 6)

# Part 17: Set Up: Load algorithm sub-task tags from json file

Load the useful nodes algorithmic sub-task tags from a JSON file stored on HuggingFace

In [ ]:
main_repo_name="PhilipQuirke/VerifiedArithmetic"
print("Loading algorithm sub-tasks from HuggingFace", main_repo_name, main_fname_algorithm_json)

file_path = hf_hub_download(repo_id=main_repo_name, filename=main_fname_algorithm_json, revision="main")

cfg.useful_nodes.load_nodes(file_path)

# Part 23A: Show algorithm quanta map

Plot the "algorithm" tags generated in previous steps as a quanta map. This is an automatically generated partial explanation of the model algorithm.

Nodes with multiple tags were tagged (found) by more than one of the above task searches

In [ ]:
qt.print_algo_purpose_results(cfg)
print()

show_quanta_map( "Algorithm Purpose Per Node", True, 2, None, qt.QType.ALGO, "", qt.get_quanta_binary, 9)

In [ ]:
show_quanta_map( "Attention Behavior Per Head", True, qt.ATTN_SHADES, None, qt.QType.ATTN, "", qt.get_quanta_attention, 9, 3 * cfg.n_layers)

# Part 23B: Show known quanta per answer digit

Each of the late positions are soley focused on calculating one answer digit. Show the data have we collected on late answer digit.  



In [ ]:
for position in range(cfg.num_question_positions + 1, cfg.n_ctx() - 1):
  print("Position:", position)

  # Calculate a table of the known quanta for the specified position for each late token position
  qt.calc_maths_quanta_for_position_nodes(cfg, position)

  plt.show()

# Part 24: Show algorithm tags

Show a list of the nodes that have proved useful in calculations, together with data on the nodes behavior and algorithmic purposes.


In [ ]:
cfg.useful_nodes.print_node_tags(qt.QType.ALGO.value, "", False)

In [ ]:
mixed_model = cfg.model_name.startswith("ins1_mix_d6_l3_h4_t40K") or cfg.model_name.startswith("ins2_mix_d6_l4_h4_t40K") or cfg.model_name.startswith("ins3_mix_d6_l4_h3_t40K")

# Part 25 : Results: Test Algorithm - Addition

## Part25A : Model add_d5_l1_h3_t30K. Tasks An.SA, An.SC, An.SS

This 1-layer model cant do all addition questions. This hypothesis mirrors Paper 1. 14/15 heads have purpose assigned. 0/6 neurons have purpose assigned.

In [ ]:
# For answer digits (excluding Amax), a An.SA node is needed before the answer digit is revealed
def test_algo_sa(algo_nodes):
  for impact_digit in range(cfg.n_digits):
    cfg.test_algo_clause(algo_nodes, qt.FilterAnd(qt.FilterAlgo(qt.add_sa_tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX)))

# For answer digits (excluding Amax and A0), a An.SC node is needed before the answer digit is revealed
def test_algo_sc(algo_nodes):
  for impact_digit in range(cfg.n_digits):
    if impact_digit > 0:
      cfg.test_algo_clause(algo_nodes, qt.FilterAnd(qt.FilterAlgo(qt.add_sc_tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX)))

# For answer digits (excluding Amax, A1 and A0), a An.SS node is needed before the answer digit is revealed
def test_algo_ss(algo_nodes):
  for impact_digit in range(cfg.n_digits):
    if impact_digit > 1:
      cfg.test_algo_clause(algo_nodes, qt.FilterAnd(qt.FilterAlgo(qt.add_ss_tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX)))

In [ ]:
if cfg.model_name == "add_d5_l1_h3_t30K":

  algo_nodes = cfg.start_algorithm_test(acfg)

  test_algo_sa(algo_nodes)
  test_algo_sc(algo_nodes)
  test_algo_ss(algo_nodes)
  cfg.print_algo_clause_results()

  cfg.print_algo_purpose_results(algo_nodes)

## Part25B : Models add_d5/d6_l2_h3_t15K. Tasks An.SA, An.SC, An.SS, An.AC

These 2 and 3-layer models can do addition accurately.

- add_d6_l2_h3_t15K: 21/31 heads have purpose assigned. 0/17 neurons have purpose assigned.

In [ ]:
# Before Amax is revealed (as a 0 or 1), there must be a Dn.C node for every digit pair
# For each digit (except A0) there must be either an An.SS or an Dn.C
def test_algo_ac_or_ss(model_nodes):
  for impact_digit in range(cfg.n_digits):
    early_ac = qt.FilterAnd(qt.FilterAlgo(qt.add_st_tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(cfg.n_digits+1), qt.QCondition.MAX))
    late_ac = qt.FilterAnd(qt.FilterAlgo(qt.add_st_tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX))
    any_ss = qt.FilterAnd(qt.FilterAlgo(qt.add_ss_tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX))

    if cfg.n_layers == 1:
      # There must be a Dn.SS node for every answer digit except A0
      if impact_digit > 0:
        cfg.test_algo_clause(model_nodes, any_ss)
    else:
      # There must be a Dn.SS node or a Dn.C node for every digit except A0
      if impact_digit > 0:
        cfg.test_algo_clause(model_nodes, qt.FilterOr(any_ss, late_ac))

      # There must a Dn.C node for every digit before the first 1 or 0 digit is calculated
      cfg.test_algo_clause(model_nodes, early_ac)

In [ ]:
if cfg.model_name == "add_d5_l2_h3_t15K" or cfg.model_name == "add_d6_l2_h3_t15K":

  algo_nodes = cfg.start_algorithm_test(acfg)

  test_algo_sa(algo_nodes)
  test_algo_sc(algo_nodes)
  test_algo_ac_or_ss(algo_nodes)
  cfg.print_algo_clause_results()

  cfg.print_algo_purpose_results(algo_nodes)

# Part 26: Results: Test Algorithm - Subtraction

## Part 26A : Model sub_d6_l2_h3_t30K. Tasks MD, MB, MZ

This 2-layer model can do subtraction accurately. TBC

In [ ]:
# For answer digits (excluding Amax), An.MD and An.MB nodes are needed before the answer digit is revealed
def test_algo_md_mb(algo_nodes):
  for impact_digit in range(cfg.n_digits):
    cfg.test_algo_clause(algo_nodes, qt.FilterAnd(qt.FilterAlgo(qt.sub_md_tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX)))

    cfg.test_algo_clause(algo_nodes, qt.FilterAnd(qt.FilterAlgo(qt.sub_mb_tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX)))


# For answer digits (excluding Amax), An.MZ nodes are needed before the answer digit is revealed
def test_algo_mz(algo_nodes):
  for impact_digit in range(cfg.n_digits):
      pass
      #cfg.test_algo_clause(algo_nodes, qt.FilterAnd(qt.FilterAlgo(sub_mz_tag(impact_digit)), qt.FilterPosition(cfg.an_to_position_name(impact_digit+1), qt.QCondition.MAX)))

In [ ]:
if cfg.model_name == "sub_d6_l2_h3_t30K":

  algo_nodes = cfg.start_algorithm_test(acfg)

  test_algo_md_mb(algo_nodes)
  test_algo_mz(algo_nodes)
  cfg.print_algo_clause_results()

  cfg.print_algo_purpose_results(algo_nodes)

## Part 26B: Test Algorithm - Subtraction - Negative Answer

To accurately predict if the answer sign is + or - the model must calculate if
D < D'. To calculate this, the model must calculate Dn < D'n or (Dn = D'n and (Dn-1 < D'n-1 or (Dn-2 = D'n-1 and ( etc. It must predict this before the answer sign is revealed.

We expect to see nodes useful in negative answer questions, with PCA bigram (or trigram) outputs, attending to these input pairs, evaluated in this order, before the answer sign is revealed.

In [ ]:
# For answer digits (excluding Amax), An.SC is needed before the answer digit is revealed
def test_algo_sc(algo_nodes):
  sc_locations = {}

  sign_position = cfg.an_to_position_name(cfg.n_digits+1)

  for impact_digit in range(cfg.n_digits):
    # For answer digits (excluding the +/- answer sign and 0 or 1 first answer digit), An.SC is calculated before the answer sign is revealed
    position = cfg.test_algo_clause(algo_nodes,  qt.FilterAnd(
      qt.FilterAlgo(qt.sub_mt_tag(impact_digit)),
      qt.FilterPosition(sign_position, qt.QCondition.MAX)))
    sc_locations[impact_digit] = position

  # Check that sc_locations[6] < sc_locations[5] < sc_locations[4] < etc
  print("SC Locations:", sc_locations)
  for impact_digit in range(cfg.n_digits):
    if impact_digit > 0:
      sc1 = sc_locations[impact_digit]
      sc2 = sc_locations[impact_digit-1]
      description = f"SC Ordering: A{impact_digit}={sc1}, A{impact_digit-1}={sc2}"
      cfg.test_algo_logic(description, sc1 >= 0 and sc2 >= 0 and sc1 < sc2)

In [ ]:
if cfg.model_name.startswith("sub_d6_l2_h3_t30K") or cfg.model_name.startswith('ins1_mix_d6_l3_h4_t40K') :

  algo_nodes = cfg.start_algorithm_test(acfg)

  test_algo_sc(algo_nodes)
  cfg.print_algo_clause_results()

  cfg.print_algo_purpose_results(algo_nodes)

# Part 27: Test Algorithm - Mixed Addition and Subtraction model

What algorithm do mixed models use to perform both addition and subtraction? Our working hypothesis is Hypothesis 2 as described in https://github.com/PhilipQuirke/verified_transformers/blob/main/mixed_model.md





## Part 27A: Calculating answer digit A2 in token position A3

The below graph displays the same (behavior and algorithm) data as the quanta maps. Refer https://github.com/PhilipQuirke/verified_transformers/blob/main/mixed_model.md section 27A for more explanation.

    


In [ ]:
qt.calc_maths_quanta_for_position_nodes(cfg, 18)
plt.show()

## Part 27.H3 Calculate whether D > D' (using NG tasks)

In [ ]:
filters = qt.FilterContains(qt.QType.MATH_NEG, "")

#print("NG tagged nodes:", qt.filter_nodes( cfg.useful_nodes, filters ).get_node_names())

show_quanta_map( "Subtraction Behavior NG Nodes", False, 2, filters, qt.QType.MATH_SUB, "", qt.get_maths_min_complexity, 9, 6)
show_quanta_map( "Attention Behavior Per NG Head", True, 10, filters, qt.QType.ATTN, "", qt.get_quanta_attention, 9, 8)
show_quanta_map( "Algorithm Purpose Per NG Node", True, 2, filters, qt.QType.ALGO, "", qt.get_quanta_binary, 9, 6)

In [ ]:
if mixed_model:
  algo_nodes = cfg.start_algorithm_test(acfg)

  test_algo_sc(algo_nodes)
  cfg.print_algo_clause_results()

  cfg.print_algo_purpose_results(algo_nodes)

# Part 30: Unit Test automated searches

In [ ]:
def check_algo_tag_exists(node_location_as_str, the_tags ):
  node_location = qt.str_to_node_location(node_location_as_str)
  node = cfg.useful_nodes.get_node(node_location)
  assert node is not None

  for the_tag in the_tags:
    if not node.contains_tag(qt.QType.ALGO.value, the_tag):
      print( "Node", node.name(), "is missing tag", the_tag, "It has tags:", node.tags )

In [ ]:
print(cfg.model_name)

if cfg.model_name.startswith('add_d6_l2_h3_t15K'):
  check_algo_tag_exists('P11L0H0', ['D2.TC'] )
  check_algo_tag_exists('P12L0H0', ['D3.TC'] )
  check_algo_tag_exists('P14L0H0', ['A5.SS', 'D4.TC'] )
  check_algo_tag_exists('P14L0H2', ['A5.SC', 'D5.TC'] )
  check_algo_tag_exists('P14L1H1', ['OPR'] )
  check_algo_tag_exists('P15L0H0', ['A4.SC'] )
  check_algo_tag_exists('P15L0H1', ['A5.SA'] )
  check_algo_tag_exists('P15L0H2', ['A5.SA'] )
  check_algo_tag_exists('P16L0H0', ['A3.SC'] )
  check_algo_tag_exists('P16L0H1', ['A4.SA'] )
  check_algo_tag_exists('P16L0H2', ['A4.SA'] )
  check_algo_tag_exists('P17L0H0', ['A2.SC'] )
  check_algo_tag_exists('P17L0H1', ['A3.SA'] )
  check_algo_tag_exists('P17L0H2', ['A3.SA'] )
  check_algo_tag_exists('P18L0H0', ['A1.SC'] )
  check_algo_tag_exists('P18L0H1', ['A2.SA'] )
  check_algo_tag_exists('P18L0H2', ['A2.SA'] )
  check_algo_tag_exists('P19L0H0', ['A0.SC'] )
  check_algo_tag_exists('P19L0H1', ['A1.SA'] )
  check_algo_tag_exists('P19L0H2', ['A1.SA'] )
  check_algo_tag_exists('P20L0H1', ['A0.SA'] )
  check_algo_tag_exists('P20L0H2', ['A0.SA'] )

if cfg.model_name.startswith('mix_d6_l3_h4_t40K'):
  check_algo_tag_exists('P8L0H1', ['OPR'] )
  check_algo_tag_exists('P13L2H0', ['A7.NG'] )
  check_algo_tag_exists('P15L0H0', ['A5.SA', 'A5.MD'] )
  check_algo_tag_exists('P15L0H3', ['A5.SA', 'A5.MD'] )
  check_algo_tag_exists('P16L0H3', ['A4.SA.A4', 'A4.MD.A4'] )
  check_algo_tag_exists('P17L0H1', ['A3.NG'] )
  check_algo_tag_exists('P17L0H3', ['A3.SA.A3', 'A3.MD.A3'] )
  check_algo_tag_exists('P18L0H1', ['A2.NG'] )
  check_algo_tag_exists('P18L0H3', ['A2.SA.A2', 'A2.MD.A2'] )
  check_algo_tag_exists('P19L0H1', ['A1.NG'] )
  check_algo_tag_exists('P19L0H3', ['A1.SA.A1', 'A1.MD.A1'] )
  check_algo_tag_exists('P20L0H0', ['A0.SA', 'A0.MD'] )
  check_algo_tag_exists('P20L0H3', ['A0.SA', 'A0.MD'] )
  check_algo_tag_exists('P20L2H1', ['A0.NG'] )

if cfg.model_name.startswith('ins1_mix_d6_l3_h4_t40K'):
  check_algo_tag_exists('P6L0H0', ['OPR'] )
  check_algo_tag_exists('P9L0H0', ['A4.MT'] )
  check_algo_tag_exists('P9L0H1', ['A4.ST'] )
  check_algo_tag_exists('P10L0H1', ['A2.MT'] )
  check_algo_tag_exists('P10L0H3', ['OPR'] )
  check_algo_tag_exists('P12L0H0', ['A3.MT'] )
  check_algo_tag_exists('P12L0H1', ['A3.ST'] )
  check_algo_tag_exists('P12L0H3', ['OPR'] )
  check_algo_tag_exists('P12L1H2', ['A1.MT'] )
  check_algo_tag_exists('P13L0H3', ['OPR'] )
  check_algo_tag_exists('P13L1H0', ['OPR'] )
  check_algo_tag_exists('P13L2H0', ['OPR'] )
  check_algo_tag_exists('P14L0H0', ['A5.SS', 'OPR'] )
  check_algo_tag_exists('P14L0H1', ['OPR'] )
  check_algo_tag_exists('P14L0H2', ['A5.SC', 'A5.ST', 'SGN'] )
  check_algo_tag_exists('P14L1H2', ['SGN'] )
  check_algo_tag_exists('P14L1H3', ['SGN'] )
  check_algo_tag_exists('P15L0H0', ['A4.SC'] )
  check_algo_tag_exists('P15L0H1', ['A5.SA', 'A5.MD'] )
  check_algo_tag_exists('P15L0H2', ['A5.SA', 'A5.MD'] )
  check_algo_tag_exists('P15L0H3', ['SGN'] )
  check_algo_tag_exists('P16L0H0', ['A3.SC'] )
  check_algo_tag_exists('P16L0H1', ['A4.SA', 'A4.MD', 'A4.ND'] )
  check_algo_tag_exists('P16L0H2', ['A4.SA', 'A4.MD', 'A4.ND'] )
  check_algo_tag_exists('P16L0H3', ['SGN'] )
  check_algo_tag_exists('P16L1H0', ['SGN'] )
  check_algo_tag_exists('P16L2H0', ['SGN'] )
  check_algo_tag_exists('P17L0H0', ['A2.SC'] )
  check_algo_tag_exists('P17L0H1', ['A3.SA', 'A3.MD', 'A3.ND'] )
  check_algo_tag_exists('P17L0H2', ['A3.SA', 'A3.MD', 'A3.ND'] )
  check_algo_tag_exists('P17L0H3', ['SGN'] )
  check_algo_tag_exists('P18L0H0', ['A1.SC'] )
  check_algo_tag_exists('P18L0H1', ['A2.SA', 'A2.MD', 'A2.ND'] )
  check_algo_tag_exists('P18L0H2', ['A2.SA', 'A2.MD', 'A2.ND'] )
  check_algo_tag_exists('P19L0H0', ['A0.MB.A1'] )
  check_algo_tag_exists('P19L0H1', ['A1.SA', 'A1.MD', 'A1.ND'] )
  check_algo_tag_exists('P19L0H2', ['A1.SA', 'A1.MD', 'A1.ND'] )
  check_algo_tag_exists('P20L0H1', ['A0.SA', 'A0.MD', 'A0.ND'] )
  check_algo_tag_exists('P20L0H2', ['A0.SA', 'A0.MD', 'A0.ND'] )